Для работы с MySQL из Python необходимо установить библиотеку mysql либо при помощи pip install mysql, либо скачав ее с <a href="https://dev.mysql.com/downloads/connector/python/">сайта</a>.<br>

**Задача**: залить наш корпус и метаданные в БД. Сделаем это на примере "Вопросов языкознания".

In [2]:
import mysql.connector as mysql
import os

Для того, чтобы подключиться к базе данных создаем соедиение при помощи метода mysql.connect. В него необходимо передать такие параметры как сервер, на котором стоит СУБД, базу данных (схему), с которой мы собираемся работать, логин и пароль пользователя, имеющего право работать с этой базой.

In [3]:
con = mysql.connect(host='127.0.0.1', database='cat_scheme', user='root', password='')

Объект con теперь позволяет работать с соединением. С его помощью создадим курсор, который будет отправлять SQL-запросы и получать их результаты. Параметр dictionary=True означает, что результаты будут возвращаться в виде словаря. То есть SELECT будет возвращать нам список записей, каждая из которых представляется словарем. Ключами в этом словаре будут имена полей, к которым будут привязаны соответствующие значения.

In [4]:
cur=con.cursor(dictionary=True)

In [13]:
DOMAINS = {
    'Лингвистика':'1'
}
JOURNALS = {
    'Вопросы языкознания':'1'
}
def write_to_db_metas(directory):
    list_of_files = os.listdir(directory)
    for file in list_of_files:
        with open(directory+'/'+file,'r',encoding='utf-8') as f:
            text = f.read()           
            text = text.replace("'", "\\'")
            lines = text.splitlines()
            metadata = []
            for line in lines:
                if line !='':
                    metadata.append(line)
            to_insert = "INSERT INTO meta_cat (domain, year, author, journal, source, Article_name) VALUES ('"+DOMAINS[metadata[4]]+"',\
            '"+metadata[3]+"','"+metadata[2]+"','"+JOURNALS['Вопросы языкознания']+"','"+metadata[0]+"','"+metadata[1]+"')"
            try:
                cur.execute(to_insert)
            except:
                print(to_insert)
            con.commit()
            
write_to_db_metas('meta2015')

In [ ]:
def write_to_db_words(directory):
    list_of_files = os.listdir(directory)
    n_file=1
    i=0
    for file in list_of_files:
        with open(directory+'/'+file,'r',encoding='utf-8') as f:
            text = f.read()           
            text = text.replace("'", "\\'")
            lines = text.splitlines()
            for line in tqdm(lines):
                if line !='':
                    spl_line = line.split('\t')
                    to_insert = "INSERT INTO words_cat (id_sent, id_text, word, lemma, POS_tag, Morph, root_dist, sint_role) VALUES ('"+spl_line[0]+"', '"+str(n_file)+"', '"+spl_line[1]+"', '"+spl_line[2]+"','"+spl_line[3]+"', '"+spl_line[4]+"', '"+spl_line[6]+"', '"+spl_line[7]+"')"
                    try:
                        cur.execute(to_insert)
                    except:
                        print(to_insert)
                    i+=1
                    con.commit()
        n_file+=1
DIR_WORDS = 'Лингвистика'
write_to_db_words(DIR_WORDS)

Теперь попробуем прочитать записанные данные. С оператором <a href="https://ru.wikipedia.org/wiki/Select_(SQL)">SELECT</a> мы уже знакомы. <i>LIMIT 0, 10</i> говорит СУБД, что вернуть надо 10 записей начиная с 0-й. При следующем запросе можно попросить вернуть следующую порцию записей (например, <i>LIMIT 0, 10</i> вернет 10 записей начиная с 10-й).

In [12]:
res=cur.execute("SELECT * FROM meta_cat LIMIT 0, 10")

Теперь необходимо получить данные с сервера. <a href="https://dev.mysql.com/doc/connector-python/en/connector-python-api-mysqlcursor-fetchall.html">fetchall</a> получает все результаты запроса. В качестве альтернативы можно использовать получение записей по одной - <a href="https://dev.mysql.com/doc/connector-python/en/connector-python-api-cext-fetch-row.html">fetch_row</a>.

In [13]:
data=cur.fetchall()

Посмотрим что у нас получилось в результате запроса. Да, в самом деле первые 10 добавленных статей.<br>
Ура! Мы умеем сохранять данные в СУБД и получать их обратно.

In [14]:
data

[{'Article_name': 'Видообразование русского глагола: префиксация и/или суффиксация?',
  'author': 'Е.В. Горбова',
  'domain': '1',
  'id_text': 1,
  'journal': '1',
  'source': 'http://www.issuesinlinguistics.ru/pubs/видообразование-русского-глагола-префиксация-и-или-суффиксация',
  'year': 2015},
 {'Article_name': 'Синтаксис «Слова о полку Игореве» в сопоставлении с его поэтическими переложениями',
  'author': 'Н.В. Патроева',
  'domain': '1',
  'id_text': 2,
  'journal': '1',
  'source': 'http://www.issuesinlinguistics.ru/pubs/синтаксис-слова-о-полку-игореве-в-сопоставлении-с-его-поэтическими-переложениями',
  'year': 2015},
 {'Article_name': 'Обзор лингвистических проектов, поддерживаемых Российским гуманитарным научным фондом в 2014 году',
  'author': 'Е.Р. Добрушина',
  'domain': '1',
  'id_text': 3,
  'journal': '1',
  'source': 'http://www.issuesinlinguistics.ru/pubs/обзор-лингвистических-проектов-поддерживаемых-российским-гуманитарным-научным-фондом-в-2014-году',
  'year': 2015